In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/nfl_scores_processed.csv')

# drop the 'schedule_date', 'weather_detail' and 'over_under_line' column
df = df.drop(['schedule_date', 'weather_detail', 'over_under_line'], axis=1)

# drop all rows with 'winner' = tie
df = df[df['winner'] != 'tie']

# create label encodings
team_names = df['team_home'].unique()
# sort team names alphabetically
team_names.sort()
# create a dict with team names as keys and ints from 1 to 32 as values
team_name_encoder = {}
for i in range(len(team_names)):
    team_name_encoder[team_names[i]] = i + 1

# home/away encoding
home_away_encoder = {'home': 0, 'away': 1 ,'PICK': 0.5}

# win/loss encoding
win_loss_encoder = {'win': 1, 'loss': 0, 'tie': 0.5}

# create reverse encoders
team_name_decoder = {v: k for k, v in team_name_encoder.items()}
home_away_decoder = {v: k for k, v in home_away_encoder.items()}
win_loss_decoder = {v: k for k, v in win_loss_encoder.items()}

# week replacements for postseason
df['schedule_week'] = df['schedule_week'].replace('Wildcard', '1')
df['schedule_week'] = df['schedule_week'].replace('Division', '2')
df['schedule_week'] = df['schedule_week'].replace('Conference', '3')
df['schedule_week'] = df['schedule_week'].replace('Superbowl', '4')

# convert the 'schedule_week' column to int
df['schedule_week'] = df['schedule_week'].astype(int)

# replace team names with ints in the df
df['team_home'] = df['team_home'].replace(team_name_encoder)
df['team_away'] = df['team_away'].replace(team_name_encoder)

# convert the 'team_favorite_id' column to 0 (home), 1 (away), 0.5 (PICK)
df['team_favorite_id'] = df['team_favorite_id'].replace(home_away_encoder)

# convert the 'winner' column to 0 (home) or 1 (away), 0.5 (tie)
df['winner'] = df['winner'].replace(home_away_encoder)

# convert the home_1, ..., home_10 columns to 0 (loss), 1 (win) or 0.5 (tie), same for away
for i in range(1, 11):
    df['home_' + str(i)] = df['home_' + str(i)].replace(win_loss_encoder)
    df['away_' + str(i)] = df['away_' + str(i)].replace(win_loss_encoder)

# replace 'NaN' string with np.nan
df = df.replace('NaN', np.nan)

# for all weather columns, fill the empty strings with the mean of the column
df['weather_temperature'].fillna(df['weather_temperature'].mean(), inplace=True)
df['weather_wind_mph'].fillna(df['weather_wind_mph'].mean(), inplace=True)
df['weather_humidity'].fillna(df['weather_humidity'].mean(), inplace=True)

df['weather_temperature'] = df['weather_temperature'].replace('', df['weather_temperature'].mean())
df['weather_wind_mph'] = df['weather_wind_mph'].replace('', df['weather_wind_mph'].mean())
df['weather_humidity'] = df['weather_humidity'].replace('', df['weather_humidity'].mean())

# drop all rows with 'NaN' values (games for which fewer than 10 games of history are available)
df = df.dropna()

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('winner', axis=1), df['winner'], test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# create a random forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100, max_depth=10)

# fit the model on the training data
rf.fit(X_train, y_train)

# predict on the test data
y_pred = rf.predict(X_test)

# calculate accuracy on the test data
accuracy = (y_pred == y_test).mean()
print(f'Test accuracy: {accuracy:.3f}')


GENERATE DEPENDENCE PLOTS

In [ ]:
from sklearn.inspection import plot_partial_dependence
import matplotlib.pyplot as plt

feature_of_interest = 'team_favorite_id'
fig, ax = plt.subplots()
plot_partial_dependence(rf, X_test, features=[feature_of_interest], ax=ax)
plt.show()

# Generate partial dependence plots for multiple features (interaction)
#features_of_interest = [('feature1', 'feature2')]
#fig, ax = plt.subplots()
#plot_partial_dependence(rf, X_test, features=features_of_interest, ax=ax)
#plt.show()